In [1]:
# -*- coding: utf-8
# Transparency International - Brazil (https://transparenciainternacional.org.br)
# Reinaldo Chaves (@paidatocandeira)
# Faz filtros e somas com os processos de políticos
#

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

Arquivo da raspagem, já preechido manualmente com os dados dos processos

In [2]:
df_processos = pd.read_excel('processos/folha/processos_parlamentares_revisao_reinaldo.xlsx',sheet_name='Planilha1', 
                            converters={'cpf': lambda x: str(x)})

In [3]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1417 entries, 0 to 1416
Data columns (total 27 columns):
cpf                                1136 non-null object
nome                               1417 non-null object
nome_completo                      1417 non-null object
partido                            1414 non-null object
uf                                 1414 non-null object
cargo_parlamentar                  1417 non-null object
tipo                               1180 non-null object
classe                             1417 non-null object
numero                             1417 non-null object
único                              1381 non-null object
assunto                            1414 non-null object
data_inicial                       1417 non-null object
data_final                         1417 non-null object
andamento                          1416 non-null object
link                               1417 non-null object
transparencia                      1415 non-null obje

In [4]:
df_selecao_atual = df_processos[(df_processos['classe'] != 'Nope')]
df_selecao_atual = df_selecao_atual[pd.notnull(df_selecao_atual['tipo'])]

In [5]:
df_selecao_atual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1135 entries, 2 to 1415
Data columns (total 27 columns):
cpf                                1134 non-null object
nome                               1135 non-null object
nome_completo                      1135 non-null object
partido                            1132 non-null object
uf                                 1132 non-null object
cargo_parlamentar                  1135 non-null object
tipo                               1135 non-null object
classe                             1135 non-null object
numero                             1135 non-null object
único                              1099 non-null object
assunto                            1132 non-null object
data_inicial                       1135 non-null object
data_final                         1135 non-null object
andamento                          1134 non-null object
link                               1135 non-null object
transparencia                      1133 non-null obje

In [6]:
df_selecao_atual.to_excel('processos/folha/processos_dia4setembro.xlsx',sheet_name='Sheet1')

# Acessa candidatos

In [6]:
candidatos_2018_todos = pd.read_excel('processos/folha/candidatos_2018_todos.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x), 
                                                                                              'cpf': lambda x: str(x),  
                                                                                              'numero_urna': lambda x: str(x),
                                                                                              'titulo_eleitor': lambda x: str(x),     
                                                                                                  })

In [8]:
candidatos = pd.read_excel('/home/reinaldo/Documentos/Code/ti/repo/besouro/candidatos_bens_padrao/base_padrao_dia17set.xlsx',sheet_name='Sheet1', converters={'cpf': lambda x: str(x),  
                                                                                              'numero_urna': lambda x: str(x),
                                                                                              'titulo_eleitor': lambda x: str(x)     
                                                                                                  })

In [7]:
candidatos_2018_todos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28063 entries, 0 to 435
Data columns (total 16 columns):
uf                      28063 non-null object
cargo                   28063 non-null object
sequencial              28063 non-null object
numero_urna             28063 non-null object
nome_completo           28063 non-null object
nome_urna               28063 non-null object
cpf                     28063 non-null object
e_mail                  28063 non-null object
situacao_candidatura    28063 non-null object
partido_eleicao         28063 non-null object
uf_nascimento           28063 non-null object
municipio_nascimento    28063 non-null object
data_nascimento         28063 non-null object
titulo_eleitor          28063 non-null object
ocupacao                28063 non-null object
reeleicao               28063 non-null object
dtypes: object(16)
memory usage: 3.6+ MB


In [9]:
processos_candidatos = pd.merge(candidatos_2018_todos, df_selecao_atual, left_on='cpf', right_on='cpf')

In [10]:
processos_candidatos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009 entries, 0 to 1008
Data columns (total 42 columns):
uf_x                               1009 non-null object
cargo                              1009 non-null object
sequencial                         1009 non-null object
numero_urna                        1009 non-null object
nome_completo_x                    1009 non-null object
nome_urna                          1009 non-null object
cpf                                1009 non-null object
e_mail                             1009 non-null object
situacao_candidatura               1009 non-null object
partido_eleicao                    1009 non-null object
uf_nascimento                      1009 non-null object
municipio_nascimento               1009 non-null object
data_nascimento                    1009 non-null object
titulo_eleitor                     1009 non-null object
ocupacao                           1009 non-null object
reeleicao                          1009 non-null obje

Só candidatos parlamentares

In [9]:
processos_candidatos2 = pd.merge(candidatos, df_selecao_atual, left_on='cpf', right_on='cpf')

In [10]:
processos_candidatos2 = processos_candidatos2[(processos_candidatos2['interessa'] == 'sim')]

In [11]:
processos_candidatos2.to_excel('processos/folha/processos_em_atividade.xlsx',sheet_name='Sheet1')

# Quantos são réus - candidatos

In [21]:
candidatos_inq = processos_candidatos[(processos_candidatos['tipo'] == 'INQ')]
candidatos_ap = processos_candidatos[(processos_candidatos['tipo'] == 'AP')]

In [22]:
candidatos_inq = candidatos_inq[(candidatos_inq['situacao'] == 'Denúncia aceita - AP ainda não julgada') |
                                          (candidatos_inq['situacao'] == 'Denúncia recebida parcialmente - AP ainda não julgada') |
                                          (candidatos_inq['situacao'] == 'Denúncia aceita - AP julgada e condenado') 
                                         ]

In [23]:
conta1 = candidatos_inq.groupby(['tipo','cpf']).size().reset_index()
conta1.columns = ['tipo', 'cpf', 'contagem']

In [24]:
conta1

,tipo,cpf,contagem
0,INQ,00018031234,3
1,INQ,00441392415,2
2,INQ,00834939134,1
3,INQ,02906287172,1
4,INQ,04240952387,1
5,INQ,06015530553,1
6,INQ,08781460910,1
7,INQ,09582819472,1
8,INQ,09604332287,1
9,INQ,13512951287,1


In [25]:
candidatos_ap1 = candidatos_ap[(candidatos_ap['situacao'] == 'Em andamento') |
                               (candidatos_ap['situacao'] == 'Condenado') |
                              candidatos_ap['situacao'].str.contains("Ausência de foro", na=False)]

In [26]:
conta2 = candidatos_ap1.groupby(['tipo','cpf']).size().reset_index()
conta2.columns = ['tipo', 'cpf', 'contagem']

In [27]:
conta2

,tipo,cpf,contagem
0,AP,00018031234,1
1,AP,00834939134,1
2,AP,01252958803,1
3,AP,01266637699,1
4,AP,02621380425,1
5,AP,02906287172,1
6,AP,04064992172,2
7,AP,04240952387,1
8,AP,05017868488,1
9,AP,06015530553,1


In [28]:
soma =  conta1.append(conta2)

In [29]:
conta4 = soma.groupby(['cpf']).size().reset_index()
conta4.columns = ['cpf', 'contagem']
conta4

,cpf,contagem
0,00018031234,2
1,00441392415,1
2,00834939134,2
3,01252958803,1
4,01266637699,1
5,02621380425,1
6,02906287172,2
7,04064992172,1
8,04240952387,2
9,05017868488,1


In [30]:
conta4_nomes = pd.merge(conta4, processos_candidatos.drop_duplicates('cpf'), left_on='cpf', right_on='cpf')

In [31]:
conta4_nomes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 43 columns):
cpf                                53 non-null object
contagem                           53 non-null int64
uf_x                               53 non-null object
cargo                              53 non-null object
sequencial                         53 non-null object
numero_urna                        53 non-null object
nome_completo_x                    53 non-null object
nome_urna                          53 non-null object
e_mail                             53 non-null object
situacao_candidatura               53 non-null object
partido_eleicao                    53 non-null object
uf_nascimento                      53 non-null object
municipio_nascimento               53 non-null object
data_nascimento                    53 non-null object
titulo_eleitor                     53 non-null object
ocupacao                           53 non-null object
reeleicao                         

In [32]:
conta4_nomes = pd.merge(conta4, processos_candidatos, left_on='cpf', right_on='cpf')

In [33]:
conta4_nomes.to_excel('processos/folha/processos_reus_4setembro.xlsx',sheet_name='Sheet1')

# Quantos foram para outra instância

In [37]:
mask = processos_candidatos['situacao'].str.contains("Ausência de foro", na=False)

In [38]:
ausencia_foro = processos_candidatos[mask]

In [39]:
ausencia_foro.to_excel('processos/folha/processos_foram_outras_instancias.xlsx',sheet_name='Sheet1')

# Extinção da punibilidade

In [111]:
extincao = processos_candidatos[(processos_candidatos['situacao'] == 'Extinção da punibilidade')]
extincao.info()
extincao['nome_urna'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 41 to 984
Data columns (total 41 columns):
uf_x                               80 non-null object
cargo                              80 non-null object
sequencial                         80 non-null object
numero_urna                        80 non-null object
nome_completo_x                    80 non-null object
nome_urna                          80 non-null object
cpf                                80 non-null object
e_mail                             80 non-null object
situacao_candidatura               80 non-null object
partido_eleicao                    80 non-null object
uf_nascimento                      80 non-null object
municipio_nascimento               80 non-null object
data_nascimento                    80 non-null object
titulo_eleitor                     80 non-null object
ocupacao                           80 non-null object
reeleicao                          80 non-null object
nome                           

JADER BARBALHO                8
NILSON LEITÃO                 6
VANDER LOUBET                 4
GIACOBO                       3
REQUIÃO                       3
VENEZIANO                     3
NELSON MARQUEZELLI            2
ALFREDO NASCIMENTO            2
NEWTON CARDOSO JR             2
WEVERTON                      2
FLEXA RIBEIRO                 2
CABO DACIOLO                  2
ROMÁRIO FARIA                 2
VALDIR RAUPP                  2
AELTON FREITAS                1
OSMAR TERRA                   1
HERMES FRANGÃO PARCIANELLO    1
JOZI ARAUJO                   1
JULIO LOPES                   1
VALADARES                     1
JARBAS                        1
SILAS CÂMARA                  1
VALMIR ASSUNÇÃO               1
MARCO TEBALDI                 1
ZÉ MARANHÃO                   1
AGUINALDO RIBEIRO             1
CARLOS MELLES                 1
JUNJI ABE                     1
CELSO RUSSOMANNO              1
CÁSSIO CUNHA LIMA             1
PAES LANDIM                   1
DIMAS FA

# Faz contagem de tipos de acusações

Dados estão na coluna resumo separados por vírgula. Exemplo: 'Corrupção ativa, Corrupção passiva, Lavagem de dinheiro'

In [ ]:
df_conta = processos_candidatos.groupby(['cpf', 'tipo']).resumo.agg(', '.join).str.split(', ').agg(Counter).reset_index()

In [41]:
lista = []

In [42]:
for _, g in df_conta.groupby('cpf'):
    x = f'{_} tem investigações no STF ou que foram para outros tribunais sobre: '
    cpf = _
    sintese = [f'{v} {t}(s) por {k}' for d,t in zip(g.resumo, g.tipo) for (k, v) in d.items()]
    
    lista.append({'cpf': cpf, 
                  'sintese': sintese}) 
    
    lines = x + '\n'.join(sintese)
    #print(lines)

In [43]:
df_sintese = pd.DataFrame(lista, columns = ['cpf', 'sintese'])

In [44]:
def limpeza(linha):
    palavras = str(linha["sintese"])
    palavras = palavras.replace('[', '')
    palavras = palavras.replace(']', '')
    return str(palavras)

df_sintese["sintese"] = df_sintese.apply(limpeza, axis=1)

In [45]:
df_sintese_nomes = pd.merge(df_sintese, processos_candidatos.drop_duplicates('cpf'), left_on='cpf', right_on='cpf')

In [46]:
df_sintese_nomes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 0 to 302
Data columns (total 43 columns):
cpf                                303 non-null object
sintese                            303 non-null object
uf_x                               303 non-null object
cargo                              303 non-null object
sequencial                         303 non-null object
numero_urna                        303 non-null object
nome_completo_x                    303 non-null object
nome_urna                          303 non-null object
e_mail                             303 non-null object
situacao_candidatura               303 non-null object
partido_eleicao                    303 non-null object
uf_nascimento                      303 non-null object
municipio_nascimento               303 non-null object
data_nascimento                    303 non-null object
titulo_eleitor                     303 non-null object
ocupacao                           303 non-null object
reeleicao      

In [47]:
df_sintese_nomes = df_sintese_nomes.iloc[:,:8]
df_sintese_nomes.drop('uf_x', axis=1, inplace = True)
df_sintese_nomes.drop('cargo', axis=1, inplace = True)
df_sintese_nomes.drop('sequencial', axis=1, inplace = True)
df_sintese_nomes.drop('numero_urna', axis=1, inplace = True)
df_sintese_nomes.drop('nome_completo_x', axis=1, inplace = True)

In [48]:
df_sintese_nomes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 0 to 302
Data columns (total 3 columns):
cpf          303 non-null object
sintese      303 non-null object
nome_urna    303 non-null object
dtypes: object(3)
memory usage: 9.5+ KB


In [49]:
df_sintese_nomes.reset_index().head()

,index,cpf,sintese,nome_urna
0,0,00007487738,"'1 INQ(s) por Lavagem de dinheiro', '1 INQ(s) ...",ALTINEU CORTES
1,1,00014125153,"'1 INQ(s) por Associação criminosa', '4 INQ(s)...",EDISON LOBÃO
2,2,00018031234,"'1 AP(s) por Desvio de verbas', '1 AP(s) por F...",JADER BARBALHO
3,3,00031226540,"'2 INQ(s) por Corrupção passiva', '1 INQ(s) po...",MÁRIO NEGROMONTE JR
4,4,00105457434,"'1 INQ(s) por Crimes Eleitorais', '1 INQ(s) po...",JARBAS


# Soma por tipo de crime

In [71]:
newdf=processos_candidatos.set_index(['cpf','tipo'])['resumo'].str.split(',').apply(pd.Series).stack().to_frame('resumo').reset_index(level=[0,1])
newdf['count']=newdf.groupby(['cpf','tipo','resumo'])['resumo'].transform('size')
newdf.head()

,cpf,tipo,resumo,count
0,96980486853,INQ,Crimes da Lei de licitações,2
0,96980486853,INQ,Crimes da Lei de licitações,2
0,96980486853,INQ,Compra de votos,1
0,96980486853,INQ,Sonegação de contribuição previdenciária,1
0,96980486853,INQ,Omissão de valor doado em campanha,1


In [51]:
newdf_nomes = pd.merge(newdf, df_sintese_nomes.drop_duplicates('cpf'), left_on='cpf', right_on='cpf')
newdf_nomes.drop('sintese', axis=1, inplace = True)

In [52]:
newdf_nomes.to_excel('processos/folha/soma_tipo_crimes.xlsx',sheet_name='Sheet1')

In [100]:
s = (processos_candidatos.pop('resumo').str.split(',', expand=True)
       .stack()
       .reset_index(level=1, drop=True)
       .rename('resumo')
       .str.strip()
    )

In [101]:
df = processos_candidatos.join(s).groupby(['cpf','tipo','resumo']).size().reset_index(name='contagem')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 4 columns):
cpf         1170 non-null object
tipo        1170 non-null object
resumo      1170 non-null object
contagem    1170 non-null int64
dtypes: int64(1), object(3)
memory usage: 36.6+ KB


In [102]:
df_nomes = pd.merge(df, df_sintese_nomes.drop_duplicates('cpf'), left_on='cpf', right_on='cpf')
df_nomes.drop('sintese', axis=1, inplace = True)
df_nomes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1170 entries, 0 to 1169
Data columns (total 5 columns):
cpf          1170 non-null object
tipo         1170 non-null object
resumo       1170 non-null object
contagem     1170 non-null int64
nome_urna    1170 non-null object
dtypes: int64(1), object(4)
memory usage: 54.8+ KB


In [103]:
df_nomes.reset_index().head(10)

,index,cpf,tipo,resumo,contagem,nome_urna
0,0,00007487738,INQ,Corrupção ativa,1,ALTINEU CORTES
1,1,00007487738,INQ,Corrupção passiva,1,ALTINEU CORTES
2,2,00007487738,INQ,Investigação Penal,1,ALTINEU CORTES
3,3,00007487738,INQ,Lavagem de dinheiro,1,ALTINEU CORTES
4,4,00007487738,INQ,Ocultação de Bens,1,ALTINEU CORTES
5,5,00007487738,INQ,Peculato,1,ALTINEU CORTES
6,6,00007487738,INQ,Quadrilha ou Bando,1,ALTINEU CORTES
7,7,00014125153,INQ,Associação criminosa,1,EDISON LOBÃO
8,8,00014125153,INQ,Corrupção ativa,1,EDISON LOBÃO
9,9,00014125153,INQ,Corrupção passiva,4,EDISON LOBÃO


In [104]:
df_nomes.to_excel('processos/folha/soma_tipo_crimes.xlsx',sheet_name='Sheet1')

In [105]:
df_nomes.groupby("resumo").contagem.sum().reset_index().sort_values("contagem", ascending=False)

,resumo,contagem
127,Lavagem de dinheiro,158
34,Corrupção passiva,156
59,Crimes da Lei de licitações,139
145,Peculato,139
136,Ocultação de Bens,100
42,Crimes Eleitorais,88
60,Crimes de Responsabilidade,73
32,Corrupção ativa,58
123,Investigação Penal,54
96,Falsidade ideológica,49


In [110]:
lavagem = df_nomes[(df_nomes['resumo'] == 'Lavagem de dinheiro')]
lavagem.groupby("nome_urna").contagem.sum().reset_index().sort_values("contagem", ascending=False)

,nome_urna,contagem
64,RENAN,12
68,ROMERO JUCÁ,8
73,VALDIR RAUPP,7
24,EDISON LOBÃO,5
23,DR. ANÍBAL,5
38,JADER BARBALHO,5
20,CIRO NOGUEIRA,4
59,PAULINHO DA FORÇA,4
65,ROBERTO,4
13,AÉCIO NEVES,3
